# FrozenLake V1
### This notebook attempts to solve the FrozenLake-V1 problem[part of OpenAI's gym environments] using Dynamic Programming
### Description of problem: 
### The "frozen lake" is either a 4x4 or 8x8 grid environment, where the agent[starting from top left grid] attempts to reach the goal[G] without falling into any holes[H]. All transition rewards=0, except for transition into terminal state[reward=1]. If agent falls into hole, it restarts from initial starting position[S]

#### Import modules 

In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import gym

#### Create environment 
#### In this case, we only consider deterministic problem[where action=down will definitely lead to transition to the state below(for example)] in a 8x8 gridworld

We start by creating 3 arrays: \
1) state_values: an array of 64 values(initialised to 0) for recording values for each state \
2) new_state_values: an array of 64 values(initialised to 0) for recording new values for each state \
3) actions: an array of 4 values[0,1,2,3](each representing LEFT,DOWN,RIGHT,UP actions]

delta is the difference between new & old state values(initialised to 0) \
theta is the min threshold for a value update to be allowed \
discount=gamma \
episodes = no. of complete training sequences

In [2]:
env = gym.make('FrozenLake-v1', desc=None,map_name="8x8", is_slippery=False, new_step_api=True)

observation, info = env.reset(seed=0, return_info=True)
state_values = np.zeros(64)
new_state_values = np.zeros(64)
actions = np.arange(4)
delta, theta, discount, episodes = 0, 0.0001, 0.9, 1000

#### Train agent  

We start by considering every state[64 in total] \
For each state, we create a list(q_values) to store q_value for every state-action pair \
For each action, we pass state & action to env.env.P to obtain a series of transitions \
For each transition, we get prob, next_state, reward, terminal \
We will use prob, next_state & reward to increment that particular state-action pair's value \

Once completed, we calculate delta(absolute difference between old state value & max(q_values) \
If delta exceeds threshold(theta), we then use it as our new state value \
After each complete sweep, we replace state_values with new_state_values

In [3]:
for episode in tqdm(range(episodes)):
    for state in range(len(state_values)):
        q_values = np.zeros(4)
        for action in range(len(actions)):
            transitions = env.env.P[state][action]
            for transition in transitions:
                prob, next_state, reward, terminal = transition
                q_values[action] += prob*(reward + discount*state_values[next_state])
        delta = np.abs(max(q_values) - state_values[state])
        if delta > theta:
            new_state_values[state] = max(q_values)
    state_values = new_state_values        
print(f"Sweep for {episodes} episodes done")        

100%|██████████| 1000/1000 [00:00<00:00, 1083.52it/s]

Sweep for 1000 episodes done


#### Optimize policy 

Policy optimization is done after completion of agent training \
Similar steps are followed, with the exception of selecting the index of the action that gives max(q_values) \
If there is a tie, we use np.random.choice to select at random \
Since index is also the action, selected index will be the optimal action for that state \
Finally, update policy for that state using optimal action 

In [4]:
optimal_policy = np.zeros(64)
for state in range(len(state_values)):
    q_values = np.zeros(4)
    for action in range(len(actions)):
        transitions = env.env.P[state][action]
        for transition in transitions:
            prob, next_state, reward, terminal = transition
            q_values[action] += prob*(reward + discount*state_values[next_state])
    optimal_action = np.random.choice([idx for idx in range(len(q_values)) if q_values[idx] == max(q_values)])
    optimal_policy[state] = optimal_action

Example of state_values, reshaped as 8x8 array

In [5]:
state_values.reshape(8,8)

array([[0.25418658, 0.28242954, 0.3138106 , 0.34867844, 0.38742049,
        0.43046721, 0.4782969 , 0.531441  ],
       [0.28242954, 0.3138106 , 0.34867844, 0.38742049, 0.43046721,
        0.4782969 , 0.531441  , 0.59049   ],
       [0.3138106 , 0.34867844, 0.38742049, 0.        , 0.4782969 ,
        0.531441  , 0.59049   , 0.6561    ],
       [0.34867844, 0.38742049, 0.43046721, 0.4782969 , 0.531441  ,
        0.        , 0.6561    , 0.729     ],
       [0.3138106 , 0.34867844, 0.38742049, 0.        , 0.59049   ,
        0.6561    , 0.729     , 0.81      ],
       [0.28242954, 0.        , 0.        , 0.59049   , 0.6561    ,
        0.729     , 0.        , 0.9       ],
       [0.3138106 , 0.        , 0.4782969 , 0.531441  , 0.        ,
        0.81      , 0.        , 1.        ],
       [0.34867844, 0.38742049, 0.43046721, 0.        , 0.81      ,
        0.9       , 1.        , 0.        ]])

#### Rendering 

This is where we observe our trained agent in action \
Notice how our agent is able to find an optimal path to the goal, while avoiding all holes

In [6]:
env = gym.make('FrozenLake-v1', desc=None,map_name="8x8", is_slippery=False, render_mode='human')
observation, info = env.reset(seed=0, return_info=True)
env.render()
for episode in range(100):
    state=observation
    observation, reward, done, info = env.step(action = int(optimal_policy[state]))
    env.render()
    
    if done:  
        break
        
env.close()         

/opt/anaconda3/lib/python3.8/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/opt/anaconda3/lib/python3.8/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


#### Conclusion 

As demonstrated, the usage of dynamic programming is able to find an optimal solution for this simple 8x8 gridworld problem \
However, do note that DP is seldom used due to it's computational complexity although it is important to understand it's concepts since they will be further developed upon in other methods